---
title: "Basic C++"
date: "12/25/2025"
categories: [C++]
format:
  html:
    code-line-numbers: true
    reference-location: margin
    citation-location: margin
css: ../../styles/jupyter_styles.css
---

In this blog post I will compile notes of basic C++ programming. My goal is to supplement my Parallel C++ blog post with more details on C++ programming language itself. In this way, I can keep the Parallel C++ blog post focused on parallel programming concepts, and refer to this blog post for C++ language details. I will cover only the basic C++ concepts that are necessary for understanding parallel programming in C++.

Note that I do not intend to write a complete C++ tutorial here. There are many great C++ tutorials available online. If you are new to C++, [learncpp.com](https://www.learncpp.com/) is an excellent resource with detailed tutorials which are easy to follow. If you are looking for a C++ reference, [cppreference.com](https://en.cppreference.com/w/) is one of the best C++ references available online.

I am using Jupyter notebook to write this blog post, as it allows me to keep the text, code and commands in one place, and compile it to this HTML page using [Quarto](https://quarto.org/). I developing in a Linux environment using Windows Subsystem for Linux (WSL)[^WSL]. Note that some commands may be specific to Linux environment, and may need to be adapted for other environments. 

[^WSL]: For details on WSL: [https://learn.microsoft.com/en-us/windows/wsl](https://learn.microsoft.com/en-us/windows/wsl)

## Getting Started

I start by compiling a simple C++ program to ensure that my development environment is set up correctly. I create a file named `hello.cpp` stored in the subdirectory `source`. I write this file using IPython magic command `%%writefile` as follows:

In [ ]:
%%writefile source/hello.cpp 
#include <iostream>
int main() {
    std::cout << "Hello, World!" << std::endl;
    return 0;
}

I am including the header `<iostream>`[^iostream] from the C++ standard library. It gives my program the definition of `std::cout` which I use print to the CLI, and `std::endl` which I use to print a newline character. `std` here refers to a namespace, which is a container that collects names of variables, classes, functions, etc., that are defined within it.

[^iostream]: For details on `<iostream>`: [https://en.cppreference.com/w/cpp/header/iostream.html](https://en.cppreference.com/w/cpp/header/iostream.html)

**C++ a compiled language**. Here, before I can run the program, I need to compile it using a C++ compiler. The compilation process has multiple steps. These include: 

1. `Pre-processing`: handles lines that start with `#`, such as `#include` and `#define`. 
2. `Compiling`: translates the pre-processed code to assembly code.
3. `Assembling`: translates the assembly code to machine code, producing object files.
4. `Linking`: links the object files to produce the final executable program.

Machine code is low-level instructions that the computer's CPU can execute directly and takes the form of binary code (0s and 1s). Machine code is specific to the architecture of the CPU (e.g., x86, ARM), hence a program compiled for one architecture may not run on another. 

Assembly code is a low-level programming language that is one step above machine code. It uses human-readable mnemonics to represent machine instructions which corresponds directly to a specific machine code instruction for a given CPU architecture. Hence, assembly code is also architecture-specific.

C++ programming language provides an abstraction over assembly and machine code that is portable across different CPU architectures. A C++ compiler translates C++ code to assembly and machine code specific to the target architecture during the compilation process.

I use the `g++` compiler to compile C++ programs. It is part of the GNU Compiler Collection (GCC)[^GCC_compilers] and is widely used for compiling C++ code. I compile and run the `hello.cpp` program as follows:

[^GCC_compilers]: For details on GCC: [https://gcc.gnu.org/](https://gcc.gnu.org/)

In [ ]:
%%sh
g++ source/hello.cpp -o build/hello 
./build/hello
echo $?

I am doing 3 things here:

- **Line 2**: I ask the `g++` to take the file `source/hello.cpp` and output `build/hello` after the full compilation process. 
- **Line 3**: I execute the program, and it prints to the CLI "Hello, World!". 
- **Line 4**: I check the return code of the program, which in this case is integer `0` as I defined in our program.

Next, it will be instructive to view the output of the complication process after each step. This can be done by stopping the compilation process after each step using the appropriate flags. I can view a list of available flag options using the command `g++ --help`. First, I will view the output after the pre-processing stage, as follows:

In [ ]:
%%sh
g++ source/hello.cpp -o build/hello.ii -E 
head -n 20 build/hello.ii                   

Here I have 2 commands: 

- **Line 2**: I ask `g++` to take the file `source/hello.cpp` and output `build/hello.ii` after the preprocessing stage. The flag for asking `g++` to stop after preprocessing stage is `-E`. .
- **Line 3**: I am displaying only the first 20 lines of the output file, as the file is very long. I recommend, taking a look at the file on GitHub repository. 

The output shows that the `#include <iostream>` statement in the C++ code is replaced with the contents of the header file. 

::: {.callout-note}
The third last line shows the definition of namespace `std`. Every name defined within the curly brackets `{}` after it belongs to the namespace `std`.
:::

Next, I will view the output after the compilation stage, as follows:


In [ ]:
%%sh
g++ source/hello.cpp -o build/hello.asm -S 
head -n 20 build/hello.asm

Again, I am showing the first 20 lines of the output file here. The rest of the file can be found on the GitHub repository.

- **Line 2**:  I ask `g++` to take the file `source/hello.cpp` and output `build/hello.asm` after the compiling stage. The flag for asking `g++` to stop after the compiling stage is `-S`.

The assembly code is specific to the architecture of the machine it is being compiled on. In my case, it is x86-64 architecture.

Note that if I have only the executable file and not the source file, I can still view the assembly code by disassembling the executable file, as shown below:

In [ ]:
%%sh
objdump -d -M intel build/hello > build/hello_2.asm
head -n 20 build/hello_2.asm

I use `objdump` utility available in Linux for disassembling executable files.

- **Line 2**: I ask `objdump` to take the file machine code `source/hello` and output assembly code `build/hello_2.asm` by disassembling it. The flag for asking `objdump` to disassemble the executable section of the file is `-d`. The flag `-M` passes options to the disassembler. Here I are asking it to use Intel syntax instead of the default AT&T syntax.

Going forward, it will be helpful to automate the compilation process instead to typing the commands every time. To keep things simple, I use tasks in VS code to automate the compilation process. I create a file named `tasks.json` in the `.vscode` subdirectory of my project directory, and add the following content to it:

In [ ]:
%%writefile .vscode/tasks.json
{
    "version": "2.0.0",
    "tasks": [
        {
            "label": "build",
            "type": "shell",
            "command": "g++",
            "args": ["source/${fileBasename}", "-o", "build/${fileBasenameNoExtension}.exe"],
            "group": {
                "kind": "build",
                "isDefault": true
            }
        },
        {
            "label": "run",
            "type": "shell",
            "command": "$./build/${fileBasenameNoExtension}.exe",
            "group": {
                "kind": "test",
                "isDefault": true
            }
        }, 
        {
            "label": "clean",
            "type": "shell",
            "command": "rm",
            "args": ["-rf", "build/*"],
            "group": {
                "kind": "build",
                "isDefault": false
            }
        }, 
        {
            "label": "build and run",
            "type": "shell",
            "group": {
                "kind": "test",
                "isDefault": false
            },
            "dependsOn": ["build", "run"]
        }
    ]
}

I have created a task to `build`, `run`, and `clean` the project. The final task named `build and run` does all three steps in sequence. The `clean` task deletes all files in the `build` subdirectory. I use this in my development process, but for this blog post I will continue to show the commands in this notebook.